# 전문가를 위한 Tensorflow 2 빠른 시작
- 출처: https://www.tensorflow.org/tutorials/quickstart/advanced?hl=ko

In [1]:
import tensorflow as tf
print('Tensorflow version: ', tf.__version__)

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

Tensorflow version:  2.8.0


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test, = x_train / 255.0, x_test / 255.0

'''
[ 차원 추가 ]
- numpy: np.expand_dims(증가대상, 차원)
    차원 = -1 시에 맨 뒷부분에 차원추가
- tensorflow
    - expand_dims(증가대상, 차원)
    - 증가대상[..., tf.newaxis] => 추가하고 싶은 위치에 tf.newaxis 넣어줘야함
- .reshape()
[ 차원 축소 ]
numpy: np.squeeze(축소대상)
  예) np.squeeze(train[0])
'''
# 채널을 위한 차원 추가
x_train = x_train[..., tf.newaxis].astype('float32')
x_test = x_test[..., tf.newaxis].astype('float32')

In [3]:
# tf.data를 사용해 데이터셋을 섞고 배치 만들기
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).shuffle(10000).batch(32)

In [4]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def __call__(self, x, training=False):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# 모델 인스턴스 만들기
model = MyModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
# 모델 훈련
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True는 추론과 학습과정에서 dropout같이 다른 동작이 필요한 레이어가 있는경우에만 필요
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [8]:
# 모델 테스트
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # 다음 에폭 시작 때 metrics 초기화
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch+1}, '
        f'Loss {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result()*100}, '
        f'Test Loss {test_loss.result()}, '
        f'Test Accuracy {test_accuracy.result()*100}'
    )

Epoch 1, Loss 0.006387131754308939, Accuracy: 99.81333923339844, Test Loss 0.06557603180408478, Test Accuracy 98.48999786376953
Epoch 2, Loss 0.0049150618724524975, Accuracy: 99.83833312988281, Test Loss 0.06378890573978424, Test Accuracy 98.69999694824219
Epoch 3, Loss 0.0033905075397342443, Accuracy: 99.88166809082031, Test Loss 0.08577615767717361, Test Accuracy 98.23999786376953
Epoch 4, Loss 0.005529162939637899, Accuracy: 99.82333374023438, Test Loss 0.08494763821363449, Test Accuracy 98.43000030517578
Epoch 5, Loss 0.004237568937242031, Accuracy: 99.86666107177734, Test Loss 0.07123908400535583, Test Accuracy 98.56999969482422
